In [9]:
# ------------------------------------------------------
# 완전한 Neo4j 데이터베이스 리셋 시스템
# ------------------------------------------------------

import xml.etree.ElementTree as ET
import pandas as pd
import json
from neo4j import GraphDatabase
from pathlib import Path
import logging
from datetime import datetime
import xmltodict
import time

# 로깅 설정
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# 파일 경로
XML_FILE_PATH = Path("../data/employee_relationships.xml")

# Neo4j 연결 설정
NEO4J_CONFIG = {
    "uri": "bolt://34.227.31.16:7687",            
    "username": "neo4j",                            
    "password": "cover-site-establishment"           
}

class Neo4jManager:
    """Neo4j 데이터베이스 연결 및 관리 클래스 (완전 리셋 버전)"""
    
    def __init__(self, uri, username, password):
        self.uri = uri
        self.username = username
        self.password = password
        self.driver = None
        self._connect()
    
    def _connect(self):
        """Neo4j 연결 설정"""
        try:
            self.driver = GraphDatabase.driver(
                self.uri, 
                auth=(self.username, self.password),
                max_connection_lifetime=3600,
                max_connection_pool_size=50,
                connection_acquisition_timeout=60
            )
            logger.info("Neo4j 연결 성공")
            
            # 연결 테스트
            with self.driver.session() as session:
                result = session.run("RETURN 1")
                logger.info("Neo4j 서버 응답 확인")
                
        except Exception as e:
            logger.error(f"Neo4j 연결 실패: {str(e)}")
            raise
    
    def close(self):
        if self.driver:
            self.driver.close()
            logger.info("Neo4j 연결 종료")
    
    def nuclear_database_reset(self):
        """핵 옵션: 완전한 데이터베이스 리셋"""
        
        print("🔥 핵 옵션 데이터베이스 리셋 시작...")
        
        with self.driver.session() as session:
            
            # 1단계: 모든 관계 삭제
            print("   1단계: 모든 관계 삭제 중...")
            session.run("MATCH ()-[r]->() DELETE r")
            
            # 2단계: 모든 노드 삭제  
            print("   2단계: 모든 노드 삭제 중...")
            session.run("MATCH (n) DELETE n")
            
            # 3단계: 기존 제약조건들을 하나씩 찾아서 삭제
            print("   3단계: 기존 제약조건 삭제 중...")
            
            # 제약조건 목록 조회 후 삭제
            constraints_result = session.run("SHOW CONSTRAINTS")
            for constraint in constraints_result:
                constraint_name = constraint.get('name')
                if constraint_name:
                    try:
                        session.run(f"DROP CONSTRAINT {constraint_name}")
                        print(f"      제약조건 삭제됨: {constraint_name}")
                    except Exception as e:
                        print(f"      제약조건 삭제 실패: {constraint_name} - {str(e)}")
            
            # 4단계: 인덱스도 삭제
            print("   4단계: 기존 인덱스 삭제 중...")
            indexes_result = session.run("SHOW INDEXES")
            for index in indexes_result:
                index_name = index.get('name')
                if index_name and not index_name.startswith('btree_'):  # 시스템 인덱스 제외
                    try:
                        session.run(f"DROP INDEX {index_name}")
                        print(f"      인덱스 삭제됨: {index_name}")
                    except Exception as e:
                        print(f"      인덱스 삭제 실패: {index_name} - {str(e)}")
            
            # 5단계: 통계 업데이트 대기
            print("   5단계: 데이터베이스 통계 업데이트 대기...")
            time.sleep(3)  # Neo4j가 변경사항을 완전히 반영할 시간
            
            # 검증: 정말로 모든 것이 삭제되었는지 확인
            node_count = session.run("MATCH (n) RETURN count(n) as count").single()["count"]
            rel_count = session.run("MATCH ()-[r]->() RETURN count(r) as count").single()["count"]
            
            print(f"   🔍 삭제 후 검증: 노드 {node_count}개, 관계 {rel_count}개")
            
            if node_count == 0 and rel_count == 0:
                logger.info("🎯 핵 옵션 데이터베이스 리셋 완료 - 완전히 깨끗함!")
            else:
                logger.warning(f"⚠️  일부 데이터가 남아있음: 노드 {node_count}개, 관계 {rel_count}개")
    
    def create_constraints_and_indexes(self):
        """새로운 제약조건 및 인덱스 생성"""
        
        # 잠시 대기 (데이터베이스 리셋 후)
        time.sleep(2)
        
        constraints_queries = [
            "CREATE CONSTRAINT emp_id_unique FOR (e:Employee) REQUIRE e.employee_id IS UNIQUE",
            "CREATE CONSTRAINT dept_name_unique FOR (d:Department) REQUIRE d.name IS UNIQUE",
            "CREATE CONSTRAINT proj_id_unique FOR (p:Project) REQUIRE p.project_id IS UNIQUE"
        ]
        
        index_queries = [
            "CREATE INDEX emp_job_level FOR (e:Employee) ON (e.job_level)",
            "CREATE INDEX emp_dept FOR (e:Employee) ON (e.department)"
        ]
        
        with self.driver.session() as session:
            # 제약조건 생성
            for query in constraints_queries:
                try:
                    session.run(query)
                    logger.info(f"제약조건 생성 완료: {query}")
                except Exception as e:
                    logger.error(f"제약조건 생성 실패: {str(e)}")
                    raise
            
            # 인덱스 생성
            for query in index_queries:
                try:
                    session.run(query)
                    logger.info(f"인덱스 생성 완료: {query}")
                except Exception as e:
                    logger.warning(f"인덱스 생성 실패: {str(e)}")

# 기존 연결이 있다면 닫기
if 'neo4j_manager' in locals() and neo4j_manager:
    neo4j_manager.close()

print("🔥 완전한 데이터베이스 리셋을 위한 Neo4jManager 준비 완료")
print("이 버전은 기존 데이터를 완전히 삭제하고 처음부터 시작합니다.")

🔥 완전한 데이터베이스 리셋을 위한 Neo4jManager 준비 완료
이 버전은 기존 데이터를 완전히 삭제하고 처음부터 시작합니다.


관계 생성 로직 함수들

In [10]:
# ------------------------------------------------------
# XML 파일 파싱 및 데이터 구조 분석
# ------------------------------------------------------

def parse_xml_to_graph_data(xml_file_path):
    """XML 파일을 파싱하여 그래프 생성용 데이터 추출"""
    
    if not xml_file_path.exists():
        raise FileNotFoundError(f"XML 파일을 찾을 수 없습니다: {xml_file_path}")
    
    try:
        tree = ET.parse(xml_file_path)
        root = tree.getroot()
        
        # 데이터 저장 구조
        graph_data = {
            'employees': [],
            'departments': set(),
            'projects': [],
            'hierarchy_relationships': [],
            'collaboration_relationships': [],
            'project_relationships': []
        }
        
        # 프로젝트 레지스트리 추출
        projects_section = root.find('ProjectRegistry')
        if projects_section is not None:
            for project_elem in projects_section.findall('Project'):
                project_data = {
                    'project_id': project_elem.get('project_id'),
                    'project_name': project_elem.get('project_name', project_elem.get('project_id')),
                    'project_type': project_elem.get('project_type'),
                    'team_size': int(project_elem.get('team_size', 0)),
                    'start_date': project_elem.get('start_date'),
                    'end_date': project_elem.get('end_date'),
                    'status': project_elem.get('status'),
                    'priority': project_elem.get('priority', 'medium')
                }
                graph_data['projects'].append(project_data)
        
        # 직원 정보 및 관계 추출
        employees_section = root.find('Employees')
        if employees_section is not None:
            for emp_elem in employees_section.findall('Employee'):
                
                # 직원 기본 정보
                employee_data = {
                    'employee_id': emp_elem.get('id'),
                    'name': emp_elem.get('name'),
                    'department': emp_elem.get('department'),
                    'job_role': emp_elem.get('job_role'),
                    'job_level': int(emp_elem.get('job_level', 1)),
                    'years_at_company': int(emp_elem.get('years_at_company', 0)),
                    'risk_tier': emp_elem.get('risk_tier'),
                    'persona': emp_elem.get('persona')
                }
                graph_data['employees'].append(employee_data)
                graph_data['departments'].add(emp_elem.get('department'))
                
                # 계층 관계 추출
                hierarchy = emp_elem.find('HierarchyRelationships')
                if hierarchy is not None:
                    # 상사 관계
                    reports_to = hierarchy.find('ReportsTo')
                    if reports_to is not None:
                        hierarchy_rel = {
                            'subordinate_id': emp_elem.get('id'),
                            'manager_id': reports_to.get('manager_id'),
                            'reporting_frequency': reports_to.get('reporting_frequency'),
                            'relationship_start': reports_to.get('reporting_since', '2023-01-01')
                        }
                        graph_data['hierarchy_relationships'].append(hierarchy_rel)
                
                # 협업 관계 추출
                collab = emp_elem.find('CollaborationRelationships')
                if collab is not None:
                    for colleague_elem in collab.findall('Colleague'):
                        collab_rel = {
                            'employee1_id': emp_elem.get('id'),
                            'employee2_id': colleague_elem.get('colleague_id'),
                            'collaboration_strength': float(colleague_elem.get('collaboration_strength', 0)),
                            'interaction_frequency': float(colleague_elem.get('interaction_frequency', 0)),
                            'collaboration_type': colleague_elem.get('collaboration_type'),
                            'relationship_quality': colleague_elem.get('relationship_quality'),
                            'common_projects': int(colleague_elem.get('common_projects', 0))
                        }
                        graph_data['collaboration_relationships'].append(collab_rel)
                
                # 프로젝트 참여 관계 추출
                projects = emp_elem.find('ProjectRelationships')
                if projects is not None:
                    for project_elem in projects.findall('ProjectParticipation'):
                        project_rel = {
                            'employee_id': emp_elem.get('id'),
                            'project_id': project_elem.get('project_id'),
                            'role_in_project': project_elem.get('role_in_project'),
                            'contribution_level': float(project_elem.get('contribution_level', 0)),
                            'project_start': project_elem.get('project_start'),
                            'project_end': project_elem.get('project_end'),
                            'project_status': project_elem.get('project_status')
                        }
                        graph_data['project_relationships'].append(project_rel)
        
        # 부서 정보를 리스트로 변환
        graph_data['departments'] = [{'name': dept} for dept in graph_data['departments']]
        
        logger.info(f"XML 파싱 완료:")
        logger.info(f"  - 직원: {len(graph_data['employees'])}명")
        logger.info(f"  - 부서: {len(graph_data['departments'])}개")
        logger.info(f"  - 프로젝트: {len(graph_data['projects'])}개")
        logger.info(f"  - 계층 관계: {len(graph_data['hierarchy_relationships'])}개")
        logger.info(f"  - 협업 관계: {len(graph_data['collaboration_relationships'])}개")
        logger.info(f"  - 프로젝트 참여: {len(graph_data['project_relationships'])}개")
        
        return graph_data
    
    except Exception as e:
        logger.error(f"XML 파싱 실패: {str(e)}")
        raise

# XML 데이터 파싱 실행
graph_data = parse_xml_to_graph_data(XML_FILE_PATH)

# 샘플 데이터 미리보기
print("\n=== 파싱된 데이터 샘플 ===")
print(f"직원 샘플 (첫 3명):")
for i, emp in enumerate(graph_data['employees'][:3]):
    print(f"  {i+1}. {emp['name']} - {emp['department']} - {emp['job_role']} (Level {emp['job_level']})")

print(f"\n부서 목록:")
for dept in graph_data['departments']:
    print(f"  - {dept['name']}")

print(f"\n프로젝트 샘플 (첫 3개):")
for i, proj in enumerate(graph_data['projects'][:3]):
    print(f"  {i+1}. {proj['project_id']} - {proj['project_type']} - 팀규모: {proj['team_size']}")

INFO:__main__:XML 파싱 완료:
INFO:__main__:  - 직원: 1470명
INFO:__main__:  - 부서: 3개
INFO:__main__:  - 프로젝트: 25개
INFO:__main__:  - 계층 관계: 316개
INFO:__main__:  - 협업 관계: 1098666개
INFO:__main__:  - 프로젝트 참여: 37개



=== 파싱된 데이터 샘플 ===
직원 샘플 (첫 3명):
  1. Employee_1 - Sales - Sales Executive (Level 2)
  2. Employee_2 - Research & Development - Research Scientist (Level 2)
  3. Employee_4 - Research & Development - Laboratory Technician (Level 1)

부서 목록:
  - Human Resources
  - Sales
  - Research & Development

프로젝트 샘플 (첫 3개):
  1. PRJ_001 - cross_department - 팀규모: 6
  2. PRJ_002 - single_department - 팀규모: 7
  3. PRJ_003 - single_department - 팀규모: 7


셀 2: XML 파싱 및 데이터 추출

In [11]:
# ------------------------------------------------------
# Neo4j 노드 생성 함수들 (수정됨 - MERGE 사용으로 중복 방지)
# ------------------------------------------------------

def create_employees_nodes(neo4j_manager, employees_data):
    """직원 노드 생성 (MERGE 사용으로 중복 방지)"""
    
    query = """
    UNWIND $employees as employee
    MERGE (e:Employee {employee_id: employee.employee_id})
    SET e.name = employee.name,
        e.department = employee.department,
        e.job_role = employee.job_role,
        e.job_level = employee.job_level,
        e.years_at_company = employee.years_at_company,
        e.created_at = CASE WHEN e.created_at IS NULL THEN datetime() ELSE e.created_at END
    """
    
    try:
        with neo4j_manager.driver.session() as session:
            result = session.run(query, employees=employees_data)
            summary = result.consume()
            created = summary.counters.nodes_created
            updated = summary.counters.properties_set - summary.counters.nodes_created * 6  # 6 properties per new node
            logger.info(f"직원 노드 처리 완료: 생성 {created}개, 업데이트 {max(0, updated//6)}개")
    except Exception as e:
        logger.error(f"직원 노드 생성 실패: {str(e)}")
        raise

def create_departments_nodes(neo4j_manager, departments_data):
    """부서 노드 생성 (MERGE 사용으로 중복 방지)"""
    
    query = """
    UNWIND $departments as dept
    MERGE (d:Department {name: dept.name})
    SET d.created_at = CASE WHEN d.created_at IS NULL THEN datetime() ELSE d.created_at END
    """
    
    try:
        with neo4j_manager.driver.session() as session:
            result = session.run(query, departments=departments_data)
            summary = result.consume()
            created = summary.counters.nodes_created
            logger.info(f"부서 노드 처리 완료: 생성 {created}개")
    except Exception as e:
        logger.error(f"부서 노드 생성 실패: {str(e)}")
        raise

def create_projects_nodes(neo4j_manager, projects_data):
    """프로젝트 노드 생성 (MERGE 사용으로 중복 방지)"""
    
    query = """
    UNWIND $projects as project
    MERGE (p:Project {project_id: project.project_id})
    SET p.project_name = project.project_name,
        p.project_type = project.project_type,
        p.team_size = project.team_size,
        p.start_date = date(project.start_date),
        p.end_date = date(project.end_date),
        p.status = project.status,
        p.priority = project.priority,
        p.created_at = CASE WHEN p.created_at IS NULL THEN datetime() ELSE p.created_at END
    """
    
    try:
        with neo4j_manager.driver.session() as session:
            result = session.run(query, projects=projects_data)
            summary = result.consume()
            created = summary.counters.nodes_created
            logger.info(f"프로젝트 노드 처리 완료: 생성 {created}개")
    except Exception as e:
        logger.error(f"프로젝트 노드 생성 실패: {str(e)}")
        raise

def create_employee_department_relationships(neo4j_manager):
    """직원-부서 관계 생성 (MERGE 사용으로 중복 방지)"""
    
    query = """
    MATCH (e:Employee), (d:Department)
    WHERE e.department = d.name
    MERGE (e)-[r:WORKS_IN]->(d)
    SET r.created_at = CASE WHEN r.created_at IS NULL THEN datetime() ELSE r.created_at END
    """
    
    try:
        with neo4j_manager.driver.session() as session:
            result = session.run(query)
            summary = result.consume()
            created = summary.counters.relationships_created
            logger.info(f"직원-부서 관계 처리 완료: 생성 {created}개")
    except Exception as e:
        logger.error(f"직원-부서 관계 생성 실패: {str(e)}")
        raise

def verify_nodes_created(neo4j_manager):
    """생성된 노드 수 검증"""
    
    verification_queries = {
        "직원": "MATCH (e:Employee) RETURN count(e) as count",
        "부서": "MATCH (d:Department) RETURN count(d) as count", 
        "프로젝트": "MATCH (p:Project) RETURN count(p) as count"
    }
    
    print("\n=== 노드 생성 검증 ===")
    
    with neo4j_manager.driver.session() as session:
        for node_type, query in verification_queries.items():
            try:
                result = session.run(query)
                count = result.single()["count"]
                print(f"  {node_type} 노드: {count:,}개")
            except Exception as e:
                print(f"  {node_type} 노드 확인 실패: {str(e)}")

print("수정된 노드 생성 함수들 정의 완료:")
print("- MERGE 사용으로 중복 생성 방지")
print("- 에러 핸들링 강화")
print("- 생성/업데이트 통계 개선")

수정된 노드 생성 함수들 정의 완료:
- MERGE 사용으로 중복 생성 방지
- 에러 핸들링 강화
- 생성/업데이트 통계 개선


셀 3: Neo4j 노드 생성 함수들

In [12]:
# ------------------------------------------------------
# Neo4j 노드 생성 함수들
# ------------------------------------------------------

def create_employees_nodes(neo4j_manager, employees_data):
    """직원 노드 생성"""
    
    query = """
    UNWIND $employees as employee
    CREATE (e:Employee {
        employee_id: employee.employee_id,
        name: employee.name,
        department: employee.department,
        job_role: employee.job_role,
        job_level: employee.job_level,
        years_at_company: employee.years_at_company,
        risk_tier: employee.risk_tier,
        persona: employee.persona,
        created_at: datetime()
    })
    """
    
    with neo4j_manager.driver.session() as session:
        result = session.run(query, employees=employees_data)
        logger.info(f"직원 노드 {len(employees_data)}개 생성 완료")

def create_departments_nodes(neo4j_manager, departments_data):
    """부서 노드 생성"""
    
    query = """
    UNWIND $departments as dept
    CREATE (d:Department {
        name: dept.name,
        created_at: datetime()
    })
    """
    
    with neo4j_manager.driver.session() as session:
        result = session.run(query, departments=departments_data)
        logger.info(f"부서 노드 {len(departments_data)}개 생성 완료")

def create_projects_nodes(neo4j_manager, projects_data):
    """프로젝트 노드 생성"""
    
    query = """
    UNWIND $projects as project
    CREATE (p:Project {
        project_id: project.project_id,
        project_name: project.project_name,
        project_type: project.project_type,
        team_size: project.team_size,
        start_date: date(project.start_date),
        end_date: date(project.end_date),
        status: project.status,
        priority: project.priority,
        created_at: datetime()
    })
    """
    
    with neo4j_manager.driver.session() as session:
        result = session.run(query, projects=projects_data)
        logger.info(f"프로젝트 노드 {len(projects_data)}개 생성 완료")

def create_employee_department_relationships(neo4j_manager):
    """직원-부서 관계 생성"""
    
    query = """
    MATCH (e:Employee), (d:Department)
    WHERE e.department = d.name
    CREATE (e)-[:WORKS_IN {
        created_at: datetime()
    }]->(d)
    """
    
    with neo4j_manager.driver.session() as session:
        result = session.run(query)
        summary = result.consume()
        logger.info(f"직원-부서 관계 {summary.counters.relationships_created}개 생성 완료")

print("노드 생성 함수들 정의 완료")

노드 생성 함수들 정의 완료


셀 4: Neo4j 관계 생성 함수들

In [13]:
# ------------------------------------------------------
# Neo4j 관계 생성 함수들
# ------------------------------------------------------

def create_hierarchy_relationships(neo4j_manager, hierarchy_data):
    """계층 관계 생성 (상사-부하)"""
    
    query = """
    UNWIND $relationships as rel
    MATCH (sub:Employee {employee_id: rel.subordinate_id})
    MATCH (mgr:Employee {employee_id: rel.manager_id})
    CREATE (sub)-[:REPORTS_TO {
        reporting_frequency: rel.reporting_frequency,
        relationship_start: CASE 
            WHEN rel.relationship_start IS NOT NULL AND rel.relationship_start <> '' 
            THEN date(rel.relationship_start) 
            ELSE null 
        END,
        created_at: datetime()
    }]->(mgr)
    """
    
    with neo4j_manager.driver.session() as session:
        result = session.run(query, relationships=hierarchy_data)
        summary = result.consume()
        logger.info(f"계층 관계 {summary.counters.relationships_created}개 생성 완료")

def create_collaboration_relationships(neo4j_manager, collaboration_data):
    """협업 관계 생성"""
    
    # 중복 제거: 양방향 관계를 단방향으로 변환
    unique_collaborations = []
    seen_pairs = set()
    
    for collab in collaboration_data:
        emp1, emp2 = collab['employee1_id'], collab['employee2_id']
        pair = tuple(sorted([emp1, emp2]))
        
        if pair not in seen_pairs:
            seen_pairs.add(pair)
            unique_collaborations.append(collab)
    
    # 방향성 관계로 수정하되, 양방향으로 생성하여 무방향 효과 구현
    query = """
    UNWIND $relationships as rel
    MATCH (e1:Employee {employee_id: rel.employee1_id})
    MATCH (e2:Employee {employee_id: rel.employee2_id})
    CREATE (e1)-[:COLLABORATES_WITH {
        collaboration_strength: rel.collaboration_strength,
        interaction_frequency: rel.interaction_frequency,
        collaboration_type: rel.collaboration_type,
        relationship_quality: rel.relationship_quality,
        common_projects: rel.common_projects,
        created_at: datetime()
    }]->(e2)
    CREATE (e2)-[:COLLABORATES_WITH {
        collaboration_strength: rel.collaboration_strength,
        interaction_frequency: rel.interaction_frequency,
        collaboration_type: rel.collaboration_type,
        relationship_quality: rel.relationship_quality,
        common_projects: rel.common_projects,
        created_at: datetime()
    }]->(e1)
    """
    
    with neo4j_manager.driver.session() as session:
        result = session.run(query, relationships=unique_collaborations)
        summary = result.consume()
        logger.info(f"협업 관계 {summary.counters.relationships_created}개 생성 완료")

def create_project_participation_relationships(neo4j_manager, project_relationships_data):
    """프로젝트 참여 관계 생성"""
    
    query = """
    UNWIND $relationships as rel
    MATCH (e:Employee {employee_id: rel.employee_id})
    MATCH (p:Project {project_id: rel.project_id})
    CREATE (e)-[:PARTICIPATES_IN {
        role_in_project: rel.role_in_project,
        contribution_level: rel.contribution_level,
        project_start: CASE 
            WHEN rel.project_start IS NOT NULL AND rel.project_start <> '' 
            THEN date(rel.project_start) 
            ELSE null 
        END,
        project_end: CASE 
            WHEN rel.project_end IS NOT NULL AND rel.project_end <> '' 
            THEN date(rel.project_end) 
            ELSE null 
        END,
        project_status: rel.project_status,
        created_at: datetime()
    }]->(p)
    """
    
    with neo4j_manager.driver.session() as session:
        result = session.run(query, relationships=project_relationships_data)
        summary = result.consume()
        logger.info(f"프로젝트 참여 관계 {summary.counters.relationships_created}개 생성 완료")

def create_project_collaboration_relationships(neo4j_manager):
    """프로젝트 내 팀원 간 협업 관계 생성"""
    
    # 방향성 관계로 수정
    query = """
    MATCH (e1:Employee)-[:PARTICIPATES_IN]->(p:Project)<-[:PARTICIPATES_IN]-(e2:Employee)
    WHERE e1.employee_id < e2.employee_id
    CREATE (e1)-[:PROJECT_COLLEAGUE {
        shared_project: p.project_id,
        created_at: datetime()
    }]->(e2)
    CREATE (e2)-[:PROJECT_COLLEAGUE {
        shared_project: p.project_id,
        created_at: datetime()
    }]->(e1)
    """
    
    with neo4j_manager.driver.session() as session:
        result = session.run(query)
        summary = result.consume()
        logger.info(f"프로젝트 내 협업 관계 {summary.counters.relationships_created}개 생성 완료")

print("관계 생성 함수들 정의 완료")

관계 생성 함수들 정의 완료


셀 5: Neo4j 데이터 베이스 생성 실행

In [14]:
# ------------------------------------------------------
# 완전 안전 Neo4j 데이터베이스 생성 실행
# ------------------------------------------------------

def safe_create_employees_nodes(neo4j_manager, employees_data):
    """완전히 안전한 직원 노드 생성"""
    
    # 배치 처리로 안전하게
    batch_size = 500
    total_created = 0
    
    for i in range(0, len(employees_data), batch_size):
        batch = employees_data[i:i+batch_size]
        
        query = """
        UNWIND $employees as employee
        MERGE (e:Employee {employee_id: employee.employee_id})
        ON CREATE SET 
            e.name = employee.name,
            e.department = employee.department,
            e.job_role = employee.job_role,
            e.job_level = employee.job_level,
            e.years_at_company = employee.years_at_company,
            e.created_at = datetime()
        ON MATCH SET
            e.name = employee.name,
            e.department = employee.department,
            e.job_role = employee.job_role,
            e.job_level = employee.job_level,
            e.years_at_company = employee.years_at_company
        """
        
        with neo4j_manager.driver.session() as session:
            result = session.run(query, employees=batch)
            summary = result.consume()
            total_created += summary.counters.nodes_created
    
    logger.info(f"직원 노드 생성 완료: {total_created}개")

def safe_create_departments_nodes(neo4j_manager, departments_data):
    """완전히 안전한 부서 노드 생성"""
    
    query = """
    UNWIND $departments as dept
    MERGE (d:Department {name: dept.name})
    ON CREATE SET d.created_at = datetime()
    """
    
    with neo4j_manager.driver.session() as session:
        result = session.run(query, departments=departments_data)
        summary = result.consume()
        logger.info(f"부서 노드 생성 완료: {summary.counters.nodes_created}개")

def safe_create_projects_nodes(neo4j_manager, projects_data):
    """완전히 안전한 프로젝트 노드 생성"""
    
    query = """
    UNWIND $projects as project
    MERGE (p:Project {project_id: project.project_id})
    ON CREATE SET 
        p.project_name = project.project_name,
        p.project_type = project.project_type,
        p.team_size = project.team_size,
        p.start_date = date(project.start_date),
        p.end_date = date(project.end_date),
        p.status = project.status,
        p.priority = project.priority,
        p.created_at = datetime()
    """
    
    with neo4j_manager.driver.session() as session:
        result = session.run(query, projects=projects_data)
        summary = result.consume()
        logger.info(f"프로젝트 노드 생성 완료: {summary.counters.nodes_created}개")

def create_neo4j_graph_safe(graph_data, neo4j_config):
    """완전히 안전한 Neo4j 그래프 생성"""
    
    neo4j_manager = None
    
    try:
        # Neo4j 연결
        neo4j_manager = Neo4jManager(
            neo4j_config['uri'], 
            neo4j_config['username'], 
            neo4j_config['password']
        )
        
        # 1. 핵 옵션 데이터베이스 리셋
        print("1. 🔥 핵 옵션 데이터베이스 완전 리셋...")
        neo4j_manager.nuclear_database_reset()
        
        # 2. 새로운 제약조건 생성
        print("2. 새로운 제약조건 및 인덱스 생성...")
        neo4j_manager.create_constraints_and_indexes()
        
        # 3. 안전한 노드 생성
        print("3. 안전한 노드 생성...")
        print("   3-1. 부서 노드 생성 중...")
        safe_create_departments_nodes(neo4j_manager, graph_data['departments'])
        
        print("   3-2. 직원 노드 생성 중...")  
        safe_create_employees_nodes(neo4j_manager, graph_data['employees'])
        
        print("   3-3. 프로젝트 노드 생성 중...")
        safe_create_projects_nodes(neo4j_manager, graph_data['projects'])
        
        # 노드 생성 검증
        with neo4j_manager.driver.session() as session:
            emp_count = session.run("MATCH (e:Employee) RETURN count(e) as count").single()["count"]
            dept_count = session.run("MATCH (d:Department) RETURN count(d) as count").single()["count"]
            proj_count = session.run("MATCH (p:Project) RETURN count(p) as count").single()["count"]
            
            print(f"   📊 노드 생성 검증: 직원 {emp_count}, 부서 {dept_count}, 프로젝트 {proj_count}")
        
        # 4. 기본 관계 생성
        print("4. 직원-부서 관계 생성...")
        query = """
        MATCH (e:Employee), (d:Department)
        WHERE e.department = d.name
        MERGE (e)-[:WORKS_IN]->(d)
        """
        with neo4j_manager.driver.session() as session:
            result = session.run(query)
            summary = result.consume()
            print(f"   직원-부서 관계: {summary.counters.relationships_created}개 생성")
        
        # 5. 계층 관계 생성
        if graph_data.get('hierarchy_relationships'):
            print(f"5. 계층 관계 {len(graph_data['hierarchy_relationships'])}개 생성...")
            create_hierarchy_relationships(neo4j_manager, graph_data['hierarchy_relationships'])
        
        # 6. 협업 관계 생성 (매우 조심스럽게)
        if graph_data.get('collaboration_relationships'):
            collab_data = graph_data['collaboration_relationships']
            total_relations = len(collab_data)
            print(f"6. 협업 관계 {total_relations:,}개 생성 시작...")
            
            # 작은 배치로 안전하게 처리
            batch_size = 2000  # 더 작은 배치 크기
            successful_batches = 0
            
            for i in range(0, total_relations, batch_size):
                batch = collab_data[i:i+batch_size]
                batch_num = (i // batch_size) + 1
                total_batches = (total_relations + batch_size - 1) // batch_size
                
                try:
                    create_collaboration_relationships(neo4j_manager, batch)
                    successful_batches += 1
                    
                    if batch_num % 50 == 0:  # 50배치마다 진행 상황 출력
                        progress = (batch_num / total_batches) * 100
                        print(f"   배치 진행: {batch_num}/{total_batches} ({progress:.1f}%)")
                        
                except Exception as e:
                    logger.error(f"협업 관계 배치 {batch_num} 실패: {str(e)}")
                    # 실패해도 계속 진행
                    continue
            
            print(f"   협업 관계 생성 완료: {successful_batches}/{total_batches} 배치 성공")
        
        # 7. 프로젝트 참여 관계
        if graph_data.get('project_relationships'):
            print(f"7. 프로젝트 참여 관계 {len(graph_data['project_relationships'])}개 생성...")
            create_project_participation_relationships(neo4j_manager, graph_data['project_relationships'])
        
        # 최종 통계
        print("\n🎯 그래프 생성 완료! 최종 통계:")
        with neo4j_manager.driver.session() as session:
            stats = {
                "총 노드": session.run("MATCH (n) RETURN count(n) as count").single()["count"],
                "총 관계": session.run("MATCH ()-[r]->() RETURN count(r) as count").single()["count"],
                "직원": session.run("MATCH (e:Employee) RETURN count(e) as count").single()["count"],
                "부서": session.run("MATCH (d:Department) RETURN count(d) as count").single()["count"],
                "프로젝트": session.run("MATCH (p:Project) RETURN count(p) as count").single()["count"]
            }
            
            for key, value in stats.items():
                print(f"   {key}: {value:,}개")
        
        return neo4j_manager
        
    except Exception as e:
        logger.error(f"그래프 생성 실패: {str(e)}")
        if neo4j_manager:
            neo4j_manager.close()
        raise

# 🚀 실제 실행
print("🚀 완전 안전 Neo4j 그래프 생성 시작")
print("="*60)

try:
    neo4j_manager = create_neo4j_graph_safe(graph_data, NEO4J_CONFIG)
    print("\n✅ 성공! Neo4j Browser에서 확인하세요: http://localhost:7474")
    
except Exception as e:
    print(f"\n❌ 최종 실패: {str(e)}")
    print("\n🔧 추가 해결 방법:")
    print("1. Neo4j Browser에서 수동으로 'MATCH (n) DETACH DELETE n' 실행")
    print("2. Neo4j 서버 완전 재시작")
    print("3. 새 데이터베이스 생성")
    neo4j_manager = None

INFO:__main__:Neo4j 연결 성공


🚀 완전 안전 Neo4j 그래프 생성 시작


INFO:__main__:Neo4j 서버 응답 확인


1. 🔥 핵 옵션 데이터베이스 완전 리셋...
🔥 핵 옵션 데이터베이스 리셋 시작...
   1단계: 모든 관계 삭제 중...
   2단계: 모든 노드 삭제 중...
   3단계: 기존 제약조건 삭제 중...
   4단계: 기존 인덱스 삭제 중...
      인덱스 삭제됨: index_343aff4e
      인덱스 삭제됨: index_f7700477
   5단계: 데이터베이스 통계 업데이트 대기...


INFO:__main__:🎯 핵 옵션 데이터베이스 리셋 완료 - 완전히 깨끗함!


...

   배치 진행: 500/550 (90.9%)


INFO:__main__:협업 관계 3998개 생성 완료
INFO:__main__:협업 관계 3998개 생성 완료
INFO:__main__:협업 관계 3998개 생성 완료
INFO:__main__:협업 관계 3998개 생성 완료
INFO:__main__:협업 관계 3998개 생성 완료
INFO:__main__:협업 관계 3998개 생성 완료
INFO:__main__:협업 관계 3998개 생성 완료
INFO:__main__:협업 관계 3998개 생성 완료
INFO:__main__:협업 관계 3998개 생성 완료
INFO:__main__:협업 관계 3998개 생성 완료
INFO:__main__:협업 관계 3998개 생성 완료
INFO:__main__:협업 관계 3994개 생성 완료
INFO:__main__:협업 관계 3998개 생성 완료
INFO:__main__:협업 관계 3998개 생성 완료
INFO:__main__:협업 관계 4000개 생성 완료
INFO:__main__:협업 관계 3998개 생성 완료
INFO:__main__:협업 관계 3998개 생성 완료
INFO:__main__:협업 관계 3994개 생성 완료
INFO:__main__:협업 관계 3998개 생성 완료
INFO:__main__:협업 관계 3998개 생성 완료
INFO:__main__:협업 관계 3998개 생성 완료
INFO:__main__:협업 관계 3994개 생성 완료
INFO:__main__:협업 관계 3998개 생성 완료
INFO:__main__:협업 관계 3996개 생성 완료
INFO:__main__:협업 관계 4000개 생성 완료
INFO:__main__:협업 관계 3994개 생성 완료
INFO:__main__:협업 관계 3998개 생성 완료
INFO:__main__:협업 관계 4000개 생성 완료
INFO:__main__:협업 관계 3998개 생성 완료
INFO:__main__:협업 관계 3996개 생성 완료
INFO:__main__:협업 관계 3998개 생성 완료
INFO:__m

   배치 진행: 550/550 (100.0%)
   협업 관계 생성 완료: 550/550 배치 성공
7. 프로젝트 참여 관계 37개 생성...


INFO:__main__:프로젝트 참여 관계 37개 생성 완료



🎯 그래프 생성 완료! 최종 통계:
   총 노드: 1,498개
   총 관계: 2,197,959개
   직원: 1,470개
   부서: 3개
   프로젝트: 25개

✅ 성공! Neo4j Browser에서 확인하세요: http://localhost:7474


셀 6: Neo4j 쿼리 예시들

In [17]:
# ------------------------------------------------------
# 패턴 기반 Neo4j 쿼리들 (페르소나/위험도 정보 없이 행동 패턴으로 분석)
# ------------------------------------------------------

def run_pattern_based_queries(neo4j_manager):
    """패턴 기반 Neo4j 그래프 쿼리 실행"""
    
    if not neo4j_manager:
        print("Neo4j 연결이 없습니다. 먼저 그래프를 생성하세요.")
        return
    
    pattern_queries = {
        "사회적 고립 의심 직원": """
            MATCH (e:Employee)
            OPTIONAL MATCH (e)-[c:COLLABORATES_WITH]-()
            WITH e, count(c) as collab_count,
                 avg(c.collaboration_strength) as avg_strength
            WHERE collab_count <= 3 OR avg_strength < 0.4
            RETURN e.employee_id, e.name, e.department, e.job_level,
                   collab_count as collaboration_count,
                   round(coalesce(avg_strength, 0), 3) as avg_collaboration_strength
            ORDER BY collab_count, avg_strength
            LIMIT 15
        """,
        
        "관리자별 부하직원 수": """
            MATCH (manager:Employee)<-[:REPORTS_TO]-(subordinate:Employee)
            RETURN manager.employee_id as manager_id, 
                   manager.name as manager_name,
                   manager.department as department,
                   manager.job_level as manager_level,
                   count(subordinate) as subordinate_count
            ORDER BY subordinate_count DESC
            LIMIT 10
        """,
        
        "협업 품질이 낮은 직원": """
            MATCH (e:Employee)-[c:COLLABORATES_WITH]-()
            WHERE c.relationship_quality IN ['poor', 'fair']
            WITH e, count(c) as poor_relations,
                 avg(c.collaboration_strength) as avg_strength
            WHERE poor_relations >= 2 OR avg_strength < 0.3
            RETURN e.employee_id, e.name, e.department, e.job_level,
                   poor_relations as poor_relationship_count,
                   round(avg_strength, 3) as avg_collaboration_strength
            ORDER BY poor_relations DESC, avg_strength
            LIMIT 10
        """,
        
        "프로젝트 기여도가 낮은 직원": """
            MATCH (e:Employee)-[p:PARTICIPATES_IN]->(proj:Project)
            WHERE proj.status = 'active'
            WITH e, count(p) as project_count,
                 avg(p.contribution_level) as avg_contribution,
                 collect(p.role_in_project) as roles
            WHERE avg_contribution < 0.5 AND project_count > 0
            RETURN e.employee_id, e.name, e.department, e.job_level,
                   project_count, round(avg_contribution, 3) as avg_contribution_level,
                   size([r IN roles WHERE r = 'support']) as support_role_count
            ORDER BY avg_contribution, project_count DESC
            LIMIT 10
        """,
        
        "가장 활발한 협업자": """
            MATCH (e:Employee)-[r:COLLABORATES_WITH]-()
            WHERE r.collaboration_strength > 0.5
            RETURN e.employee_id, e.name, e.department, e.job_level,
                   count(r) as strong_collaboration_count,
                   round(avg(r.collaboration_strength), 3) as avg_strength,
                   size([(e)-[rc:COLLABORATES_WITH]-() WHERE rc.relationship_quality = 'excellent' | 1]) as excellent_relationships
            ORDER BY strong_collaboration_count DESC
            LIMIT 10
        """,
        
        "프로젝트별 팀 구성": """
            MATCH (p:Project)<-[rel:PARTICIPATES_IN]-(e:Employee)
            WHERE p.status = 'active'
            RETURN p.project_id, p.project_name, p.project_type,
                   collect({
                       employee_id: e.employee_id, 
                       name: e.name, 
                       department: e.department,
                       role: rel.role_in_project,
                       contribution: rel.contribution_level
                   }) as team_members
            ORDER BY p.project_id
            LIMIT 5
        """,
        
        "관리 스팬이 높은 관리자": """
            MATCH (manager:Employee)<-[:REPORTS_TO]-(sub:Employee)
            WITH manager, count(sub) as subordinate_count
            WHERE subordinate_count > 6
            OPTIONAL MATCH (manager)-[c:COLLABORATES_WITH]-()
            WHERE c.relationship_quality IN ['poor', 'fair']
            WITH manager, subordinate_count, count(c) as poor_manager_relations
            RETURN manager.employee_id, manager.name, manager.department,
                   manager.job_level, subordinate_count,
                   poor_manager_relations as poor_relationships
            ORDER BY subordinate_count DESC, poor_manager_relations DESC
            LIMIT 10
        """
    }
    
    print("=== 패턴 기반 직원 행동 분석 ===\n")
    
    with neo4j_manager.driver.session() as session:
        for query_name, query in pattern_queries.items():
            try:
                print(f"📊 {query_name}:")
                result = session.run(query)
                records = list(result)
                
                if records:
                    # 결과를 DataFrame으로 변환하여 보기 좋게 출력
                    df_result = pd.DataFrame([record.data() for record in records])
                    print(df_result.to_string(index=False, max_colwidth=50))
                else:
                    print("   결과 없음")
                    
                print("\n" + "-"*80 + "\n")
                
            except Exception as e:
                print(f"   쿼리 실행 실패: {str(e)}\n")

def create_cognita_inference_queries():
    """Cognita가 위험도를 추론할 수 있는 패턴 분석 쿼리들"""
    
    inference_queries = {
        "종합 위험도 지표 계산": """
            MATCH (e:Employee)
            
            // 협업 지표
            OPTIONAL MATCH (e)-[c:COLLABORATES_WITH]-()
            WITH e, count(c) as collab_count, 
                 avg(c.collaboration_strength) as avg_collab_strength,
                 size([(e)-[cc:COLLABORATES_WITH]-() WHERE cc.relationship_quality IN ['poor', 'fair'] | 1]) as poor_relations
            
            // 프로젝트 참여 지표  
            OPTIONAL MATCH (e)-[p:PARTICIPATES_IN]->(proj:Project)
            WHERE proj.status = 'active'
            WITH e, collab_count, avg_collab_strength, poor_relations,
                 count(p) as active_projects, avg(p.contribution_level) as avg_contribution
            
            // 위험도 점수 계산 (0-1, 높을수록 위험)
            WITH e,
                 collab_count, avg_collab_strength, poor_relations, active_projects, avg_contribution,
                 
                 // 사회적 고립 점수 (협업 관계 적을수록 높음)
                 CASE 
                     WHEN collab_count = 0 THEN 1.0
                     WHEN collab_count <= 2 THEN 0.8
                     WHEN collab_count <= 5 THEN 0.4
                     ELSE 0.1
                 END as isolation_score,
                 
                 // 협업 품질 점수 (품질 낮을수록 높음)
                 CASE
                     WHEN avg_collab_strength IS NULL THEN 0.8
                     WHEN avg_collab_strength < 0.3 THEN 0.9
                     WHEN avg_collab_strength < 0.5 THEN 0.6
                     WHEN avg_collab_strength < 0.7 THEN 0.3
                     ELSE 0.1
                 END as quality_score,
                 
                 // 관계 문제 점수
                 CASE
                     WHEN poor_relations >= 4 THEN 0.9
                     WHEN poor_relations >= 2 THEN 0.6
                     WHEN poor_relations >= 1 THEN 0.3
                     ELSE 0.0
                 END as relationship_problem_score,
                 
                 // 프로젝트 기여 점수 (기여도 낮을수록 높음)
                 CASE
                     WHEN avg_contribution IS NULL THEN 0.5
                     WHEN avg_contribution < 0.4 THEN 0.8
                     WHEN avg_contribution < 0.6 THEN 0.4
                     ELSE 0.1
                 END as contribution_score
            
            WITH e, collab_count, avg_collab_strength, poor_relations, active_projects, avg_contribution,
                 isolation_score, quality_score, relationship_problem_score, contribution_score,
                 
                 // 종합 위험도 점수 (가중 평균)
                 round((isolation_score * 0.3 + 
                       quality_score * 0.25 + 
                       relationship_problem_score * 0.25 + 
                       contribution_score * 0.2), 3) as risk_score
            
            // 위험도 등급 할당
            WITH e, collab_count, avg_collab_strength, poor_relations, active_projects, avg_contribution, risk_score,
                 CASE
                     WHEN risk_score >= 0.7 THEN 'HIGH'
                     WHEN risk_score >= 0.4 THEN 'MEDIUM'  
                     ELSE 'LOW'
                 END as inferred_risk_tier
            
            RETURN e.employee_id, e.name, e.department, e.job_level,
                   collab_count as collaborations,
                   round(coalesce(avg_collab_strength, 0), 3) as avg_collab_strength,
                   poor_relations as poor_relationships,
                   coalesce(active_projects, 0) as active_projects,
                   round(coalesce(avg_contribution, 0), 3) as avg_contribution,
                   risk_score,
                   inferred_risk_tier as predicted_risk
            ORDER BY risk_score DESC
            LIMIT 20
        """,
        
        "부서별 위험도 분포": """
            // 위에서 계산한 로직을 재사용하여 부서별 집계
            MATCH (e:Employee)
            
            OPTIONAL MATCH (e)-[c:COLLABORATES_WITH]-()
            WITH e, count(c) as collab_count, 
                 avg(c.collaboration_strength) as avg_collab_strength,
                 size([(e)-[cc:COLLABORATES_WITH]-() WHERE cc.relationship_quality IN ['poor', 'fair'] | 1]) as poor_relations
            
            OPTIONAL MATCH (e)-[p:PARTICIPATES_IN]->(proj:Project)
            WHERE proj.status = 'active'
            WITH e, collab_count, avg_collab_strength, poor_relations,
                 count(p) as active_projects, avg(p.contribution_level) as avg_contribution,
                 
                 // 간단한 위험도 계산
                 round(
                     (CASE WHEN collab_count <= 2 THEN 0.4 ELSE 0.0 END +
                      CASE WHEN avg_collab_strength < 0.5 THEN 0.3 ELSE 0.0 END +
                      CASE WHEN poor_relations >= 2 THEN 0.3 ELSE 0.0 END), 3
                 ) as risk_score
            
            WITH e.department as department, 
                 CASE WHEN risk_score >= 0.5 THEN 'HIGH' 
                      WHEN risk_score >= 0.25 THEN 'MEDIUM'
                      ELSE 'LOW' END as risk_level
            
            RETURN department,
                   count(*) as total_employees,
                   size([x IN collect(risk_level) WHERE x = 'HIGH']) as high_risk_count,
                   size([x IN collect(risk_level) WHERE x = 'MEDIUM']) as medium_risk_count,
                   size([x IN collect(risk_level) WHERE x = 'LOW']) as low_risk_count,
                   round(toFloat(size([x IN collect(risk_level) WHERE x = 'HIGH'])) / count(*), 3) as high_risk_ratio
            ORDER BY high_risk_ratio DESC
        """
    }
    
    return inference_queries

# 패턴 기반 쿼리 실행
if 'neo4j_manager' in locals() and neo4j_manager:
    run_pattern_based_queries(neo4j_manager)
    
    print("\n=== Cognita 위험도 추론 분석 ===")
    inference_queries = create_cognita_inference_queries()
    
    with neo4j_manager.driver.session() as session:
        for query_name, query in inference_queries.items():
            try:
                print(f"\n🤖 {query_name}:")
                result = session.run(query)
                records = list(result)
                
                if records:
                    df_result = pd.DataFrame([record.data() for record in records])
                    print(df_result.to_string(index=False, max_colwidth=40))
                else:
                    print("   결과 없음")
                    
            except Exception as e:
                print(f"   쿼리 실행 실패: {str(e)}")
else:
    print("Neo4j 연결이 없어 쿼리 예시를 건너뜁니다.")
    print("연결 설정 후 이 셀을 다시 실행하세요.")

=== 패턴 기반 직원 행동 분석 ===

📊 사회적 고립 의심 직원:


   결과 없음

--------------------------------------------------------------------------------

📊 관리자별 부하직원 수:
manager_id  manager_name             department  manager_level  subordinate_count
      1775 Employee_1775 Research & Development              4                  7
      1591 Employee_1591                  Sales              5                  7
      1770 Employee_1770 Research & Development              5                  7
      1938 Employee_1938                  Sales              4                  7
      1306 Employee_1306 Research & Development              5                  7
       661  Employee_661 Research & Development              4                  7
       986  Employee_986                  Sales              4                  7
       468  Employee_468 Research & Development              4                  6
       148  Employee_148        Human Resources              4                  6
      1029 Employee_1029                  Sales              5           

셀 7: Neo4j Browser 연동 및 시각화 가이드

In [16]:
# ------------------------------------------------------
# Neo4j Browser 시각화 가이드
# ------------------------------------------------------

def generate_visualization_queries():
    """Neo4j Browser에서 사용할 시각화 쿼리들"""
    
    viz_queries = {
        "전체 조직 구조 (샘플)": """
            // 한 부서의 계층 구조 시각화
            MATCH (e:Employee)-[:WORKS_IN]->(d:Department {name: 'Sales'})
            OPTIONAL MATCH (e)-[r:REPORTS_TO]->(manager)
            RETURN e, r, manager, d
            LIMIT 20
        """,
        
        "고위험 직원과 그 관계": """
            // 고위험 직원과 연결된 모든 관계
            MATCH (high_risk:Employee {risk_tier: 'High'})
            OPTIONAL MATCH (high_risk)-[r1]-(connected)
            RETURN high_risk, r1, connected
            LIMIT 50
        """,
        
        "프로젝트 협업 네트워크": """
            // 특정 프로젝트의 팀원과 그들의 협업 관계
            MATCH (p:Project {status: 'active'})<-[:PARTICIPATES_IN]-(e:Employee)
            OPTIONAL MATCH (e)-[c:COLLABORATES_WITH]-(colleague)-[:PARTICIPATES_IN]->(p)
            RETURN p, e, c, colleague
            LIMIT 30
        """,
        
        "부서간 협업 네트워크": """
            // 서로 다른 부서 직원들 간의 협업 관계
            MATCH (e1:Employee)-[:WORKS_IN]->(d1:Department)
            MATCH (e2:Employee)-[:WORKS_IN]->(d2:Department)
            MATCH (e1)-[c:COLLABORATES_WITH]-(e2)
            WHERE d1 <> d2 AND c.collaboration_strength > 0.6
            RETURN e1, e2, c, d1, d2
            LIMIT 25
        """,
        
        "위험 전파 경로 분석": """
            // 고위험 직원으로부터 2단계 이내의 관계
            MATCH (high_risk:Employee {risk_tier: 'High'})
            MATCH path = (high_risk)-[*1..2]-(connected:Employee)
            WHERE connected.risk_tier IN ['High', 'Neutral']
            RETURN path
            LIMIT 40
        """
    }
    
    print("=== Neo4j Browser 시각화 가이드 ===\n")
    
    print("1. Neo4j Browser 접속:")
    print("   http://localhost:7474 (기본 주소)")
    print("   username: neo4j")
    print("   password: [설정한 비밀번호]\n")
    
    print("2. 아래 쿼리들을 복사해서 Neo4j Browser에서 실행하세요:\n")
    
    for name, query in viz_queries.items():
        print(f"🎨 {name}:")
        print("```cypher")
        print(query.strip())
        print("```\n")
    
    print("3. 시각화 팁:")
    print("   - 노드 크기: 직급(job_level) 또는 위험도(risk_tier)에 따라 조정")
    print("   - 노드 색상: 부서별 또는 위험등급별로 구분")
    print("   - 관계 두께: 협업 강도(collaboration_strength)에 따라 조정")
    print("   - 레이아웃: Force-directed 또는 Hierarchical 레이아웃 사용")
    
    return viz_queries

# 시각화 쿼리 생성
viz_queries = generate_visualization_queries()

# Neo4j 연결 정리
if 'neo4j_manager' in locals() and neo4j_manager:
    print(f"\n=== Neo4j 그래프 생성 완료 ===")
    print(f"✅ 총 {len(graph_data['employees'])}명의 직원 데이터가 Neo4j에 저장되었습니다.")
    print(f"✅ 계층관계, 협업관계, 프로젝트 참여 관계가 모두 생성되었습니다.")
    print(f"\n🌐 Neo4j Browser에서 그래프를 시각적으로 탐색해보세요:")
    print(f"   http://localhost:7474")
    
    # 필요한 경우 연결 종료
    # neo4j_manager.close()
else:
    print("\n⚠️  Neo4j 연결 설정을 완료한 후 다시 실행하세요.")
    print("   1. Neo4j 서버 실행")
    print("   2. 인증 정보 수정")
    print("   3. 코드 재실행")

=== Neo4j Browser 시각화 가이드 ===

1. Neo4j Browser 접속:
   http://localhost:7474 (기본 주소)
   username: neo4j
   password: [설정한 비밀번호]

2. 아래 쿼리들을 복사해서 Neo4j Browser에서 실행하세요:

🎨 전체 조직 구조 (샘플):
```cypher
// 한 부서의 계층 구조 시각화
            MATCH (e:Employee)-[:WORKS_IN]->(d:Department {name: 'Sales'})
            OPTIONAL MATCH (e)-[r:REPORTS_TO]->(manager)
            RETURN e, r, manager, d
            LIMIT 20
```

🎨 고위험 직원과 그 관계:
```cypher
// 고위험 직원과 연결된 모든 관계
            MATCH (high_risk:Employee {risk_tier: 'High'})
            OPTIONAL MATCH (high_risk)-[r1]-(connected)
            RETURN high_risk, r1, connected
            LIMIT 50
```

🎨 프로젝트 협업 네트워크:
```cypher
// 특정 프로젝트의 팀원과 그들의 협업 관계
            MATCH (p:Project {status: 'active'})<-[:PARTICIPATES_IN]-(e:Employee)
            OPTIONAL MATCH (e)-[c:COLLABORATES_WITH]-(colleague)-[:PARTICIPATES_IN]->(p)
            RETURN p, e, c, colleague
            LIMIT 30
```

🎨 부서간 협업 네트워크:
```cypher
// 서로 다른 부서 직원들 간의 협업 관계
            MATCH (e1: